<h1>Neural Network!!</h1>

Since I quite already know what the neural network is, if I have a time I will write the definition later. For now, I'll represent the example of neural network implementation on each type of neural network (pytorch & tensorflow)

<h2> Type of neural network that we're going to learn today : </h2>

1. Standard Neural Network

2. Convolutional Neural Network

3. Recurrent Neural Network

<h3> Activation Function </h3>
  Before we go any further, I would like to introduce the "Activation function" that makes some output easier to compute. 
  
  1. Sigmoid Activation Function 
  
  2. Hyperbolic Tangent Activation Function (tanh)
  
  3. Rectified Linear Unit Activation Function (ReLU)
  
  4. Leaky ReLU 
  
  5. Exponential Linear Units (ELU)
  
  6. Maxout
  
  More detail here : https://medium.com/@shrutijadon10104776/survey-on-activation-functions-for-deep-learning-9689331ba092

<h3> Standard Neural Network : Implementation </h3>

In [ ]:
# This is the naive implementation : later 

In [29]:
# Tensorflow implementation
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [11]:
# Python optimisation variables
learning_rate = 0.5
epochs = 10
batch_size = 100

## This is for Tensorflow v1 only
## Placeholder is a variable that we don't assign any value yet, (like input or output, or any structure)
#x = tf.placeholder(tf.float32, [None, 784])
#y = tf.placeholder(tf.float32, [None,10])

#W1 = tf.Variable(tf.random.normal([784,30], stddev=0.03), name='W1')
#b1 = tf.constant(tf.random.normal([30]), name='b1')

#W2 = tf.Variable(tf.random.normal([30,10], stddev=0.03), name='W2')
#b2 = tf.constant(tf.random.normal([10],name='b2'))

#hidden_out = tf.add(tf.matmul(W1,x),b1)
#hidden_out = tf.nn.relu(hidden_out)
#y_ = tf.nn.softmax(tf.add(tf.matmul(W2,hidden_out),b2)) 
##### I'm gonna end tensorflow v1 here, since it's deprecated

## Tensorflow v2 (and from now on, I'll use tensorflow v2)
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28)), # Flatten the 2D to 1D 
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.2), # Randomly set input to 0 with a frequency of "rate" (in this case : 0.2)
    tf.keras.layers.Dense(10)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
             loss=loss_fn,
             metrics=['accuracy'])
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.4180 - accuracy: 0.7440
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.6229 - accuracy: 0.8364
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4867 - accuracy: 0.8689
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4242 - accuracy: 0.8870
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3905 - accuracy: 0.8973


In [12]:
model.evaluate(x_test,y_test,verbose=2) # Verbose is about how do you want to see on trainning process (Display)

313/313 - 1s - loss: 0.3343 - accuracy: 0.9376


[0.3342955410480499, 0.9376000165939331]

In [64]:
#Pytorch implementation
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Neural_Network(nn.Module):
    def __init__(self,input_size,output_size):
        super(Neural_Network,self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        self.W1 = torch.randn(self.input_size,128)  
        self.W2 = torch.randn(128,self.output_size)
    
    def forward(self,X):
        X = torch.flatten(X,start_dim=1)
        self.z = torch.matmul(X,self.W1)
        self.z2 = self.sigmoid(self.z)
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3)
        return o
    
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def backward(self,X,y,o):
        X = torch.flatten(X,start_dim=1)
        self.o_error = y-o
        self.o_delta = self.o_error*self.sigmoidPrime(o)
        self.z2_error = torch.matmul(self.o_delta,torch.t(self.W2)) # torch.t(X) is to transpose X
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X),self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2),self.o_delta)
    
    def train(self, X, y):
        # forward + backward pass for training
        X = torch.flatten(X,start_dim=1)
        o = self.forward(X)
        self.backward(X, y, o)
    
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))

X = torch.from_numpy(x_train).float()
y = [[1 if i == x else 0 for i in range(10)] for x in y_train]
y = torch.from_numpy(np.array(y)).float()
NN = Neural_Network(784,10)
print("Start trainning : ")
for i in range(1000):  # trains the NN 1,000 times
    if i % 100 == 0:
        print("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X,y)

Start trainning : 
#0 Loss: 0.43502917885780334
#100 Loss: 0.10000000149011612
#200 Loss: 0.10000000149011612
#300 Loss: 0.10000000149011612
#400 Loss: 0.10000000149011612
#500 Loss: 0.10000000149011612
#600 Loss: 0.10000000149011612
#700 Loss: 0.10000000149011612
#800 Loss: 0.10000000149011612
#900 Loss: 0.10000000149011612


Note1 : We can use torch.save(model,name) to save all weight in neural network. 

Note2 :  detach() is the method to detach from the pytorch graph, and that value will not contribute gradient in the main graph. See more explanation at : http://www.bnikolic.co.uk/blog/pytorch-detach.html#:~:text=The%20detach()%20method%20constructs,visualised%20using%20the%20torchviz%20package.

In [74]:
model = nn.Sequential(nn.Flatten(),
                      nn.Linear(784,128),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(128,10))
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.2)
optimizer.zero_grad()
for i in range(1000):  # trains the NN 1,000 times
    loss = loss_fn(model(X),y)
    if (i+1) % 100 == 0:
        print("#" + str(i+1) + " Loss: " + str(loss.item()))  # mean sum squared loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


#100 Loss: 828414272.0
#200 Loss: 235820.078125
#300 Loss: 67.22174835205078
#400 Loss: 0.10908998548984528
#500 Loss: 0.08998025208711624
#600 Loss: 0.08997682482004166
#700 Loss: 0.08997835218906403
#800 Loss: 0.08997835963964462
#900 Loss: 0.08997835963964462
#1000 Loss: 0.08997835963964462
